In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [2]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [3]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)

In [8]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


2022-02-27 09:42:11.333702: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
35/35 [==============================] - 0s 1ms/step - loss: 619.2080 - accuracy: 0.1833
Epoch 2/100
35/35 [==============================] - 0s 1ms/step - loss: 135.9603 - accuracy: 0.5880
Epoch 3/100
35/35 [==============================] - 0s 1ms/step - loss: 80.9903 - accuracy: 0.6552
Epoch 4/100
35/35 [==============================] - 0s 1ms/step - loss: 58.1002 - accuracy: 0.6788
Epoch 5/100
35/35 [==============================] - 0s 1ms/step - loss: 43.7942 - accuracy: 0.7105
Epoch 6/100
35/35 [==============================] - 0s 1ms/step - loss: 34.8264 - accuracy: 0.7214
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 28.0888 - accuracy: 0.7287
Epoch 8/100
35/35 [==============================] - 0s 1ms/step - loss: 21.8849 - accuracy: 0.7387
Epoch 9/100
35/35 [==============================] - 0s 1ms/step - loss: 15.7902 - accuracy: 0.7278
Epoch 10/100
35/35 [==============================] - 0s 1ms/step - loss: 10.9026 - accuracy: 0.73

35/35 [==============================] - 0s 770us/step - loss: 0.4628 - accuracy: 0.8285
Epoch 83/100
35/35 [==============================] - 0s 762us/step - loss: 0.6845 - accuracy: 0.8031
Epoch 84/100
35/35 [==============================] - 0s 759us/step - loss: 0.5329 - accuracy: 0.8103
Epoch 85/100
35/35 [==============================] - 0s 813us/step - loss: 1.6039 - accuracy: 0.7822
Epoch 86/100
35/35 [==============================] - 0s 763us/step - loss: 0.9780 - accuracy: 0.7840
Epoch 87/100
35/35 [==============================] - 0s 764us/step - loss: 0.9505 - accuracy: 0.7886
Epoch 88/100
35/35 [==============================] - 0s 776us/step - loss: 0.9453 - accuracy: 0.7668
Epoch 89/100
35/35 [==============================] - 0s 766us/step - loss: 0.5042 - accuracy: 0.8321
Epoch 90/100
35/35 [==============================] - 0s 754us/step - loss: 0.5747 - accuracy: 0.8113
Epoch 91/100
35/35 [==============================] - 0s 757us/step - loss: 0.6465 - accuracy: 

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.7552 - accuracy: 0.8451 - 127ms/epoch - 11ms/step
Loss: 0.7552017569541931, Accuracy: 0.845108687877655


# 6.1

In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_te)

Epoch 1/100
 1/35 [..............................] - ETA: 11s - loss: 0.6823 - accuracy: 0.5312
Epoch 1: saving model to checkpoints/weights.01.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.6435 - accuracy: 0.6125
Epoch 2/100
 1/35 [..............................] - ETA: 0s - loss: 0.5619 - accuracy: 0.7812
Epoch 2: saving model to checkpoints/weights.02.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.5459 - accuracy: 0.7595
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.4878 - accuracy: 0.6562
Epoch 3: saving model to checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.5025 - accuracy: 0.7976
Epoch 4/100
 1/35 [..............................] - ETA: 0s - loss: 0.5625 - accuracy: 0.8125
Epoch 4: saving model to checkpoints/weights.04.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.4774 - accuracy: 0.8176
Epoch 5/100
 1/35 [..............................] - ETA: 0

Epoch 36/100
 1/35 [..............................] - ETA: 0s - loss: 0.3067 - accuracy: 0.8750
Epoch 36: saving model to checkpoints/weights.36.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.2784 - accuracy: 0.8929
Epoch 37/100
 1/35 [..............................] - ETA: 0s - loss: 0.2185 - accuracy: 0.9062
Epoch 37: saving model to checkpoints/weights.37.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.2747 - accuracy: 0.8966
Epoch 38/100
 1/35 [..............................] - ETA: 0s - loss: 0.2775 - accuracy: 0.9062
Epoch 38: saving model to checkpoints/weights.38.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.2717 - accuracy: 0.8956
Epoch 39/100
 1/35 [..............................] - ETA: 0s - loss: 0.2990 - accuracy: 0.8438
Epoch 39: saving model to checkpoints/weights.39.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2690 - accuracy: 0.8984
Epoch 40/100
 1/35 [..............................] 

35/35 [==============================] - 0s 1ms/step - loss: 0.2103 - accuracy: 0.9129
Epoch 71/100
 1/35 [..............................] - ETA: 0s - loss: 0.1279 - accuracy: 0.9688
Epoch 71: saving model to checkpoints/weights.71.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2088 - accuracy: 0.9120
Epoch 72/100
 1/35 [..............................] - ETA: 0s - loss: 0.0927 - accuracy: 1.0000
Epoch 72: saving model to checkpoints/weights.72.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.2077 - accuracy: 0.9102
Epoch 73/100
 1/35 [..............................] - ETA: 0s - loss: 0.2888 - accuracy: 0.8438
Epoch 73: saving model to checkpoints/weights.73.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2062 - accuracy: 0.9111
Epoch 74/100
 1/35 [..............................] - ETA: 0s - loss: 0.1911 - accuracy: 0.9375
Epoch 74: saving model to checkpoints/weights.74.hdf5
35/35 [==============================] - 0s 1ms/step

NameError: name 'X_test_scaled' is not defined

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

NameError: name 'X_test_scaled' is not defined

In [15]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=8))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50


ValueError: in user code:

    File "/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/hasna/opt/anaconda3/envs/mlenv/lib/python3